In [1]:
pip install google-play-scraper

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/50.2 kB ? eta -:--:--
     ------------------------ --------------- 30.7/50.2 kB ? eta -:--:--
     ---------------------------------------- 50.2/50.2 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
from google_play_scraper import app, reviews, Sort
import pandas as pd
import time
import datetime
from tqdm import tqdm

# 目标 APP ID（Genshin Impact）
app_id = "com.miHoYo.GenshinImpact"

# 设定爬取的国家和语言（美国地区，英文）
target_country = "us"
target_lang = "en"

# 每次抓取的数量（最大 500）
batch_size = 500

# 设定爬取的时间范围（只爬取近 1 年）
one_year_ago = datetime.datetime.now() - datetime.timedelta(days=365)

# 数据存储路径
output_file = "genshin_impact_reviews_last_year.csv"

# 1️⃣ 获取 APP 的总评论数
app_info = app(app_id, lang=target_lang, country=target_country)
total_reviews_available = app_info["reviews"]

print(f"🎯 该 APP 在 Google Play 上的总评论数: {total_reviews_available}")

# 2️⃣ 计算大致需要爬取的次数
estimated_batches = total_reviews_available // batch_size
print(f"⏳ 预计需要爬取 {estimated_batches} 次，每次 {batch_size} 条")

# 3️⃣ 初始化 tqdm 进度条
pbar = tqdm(total=total_reviews_available, desc="Total Reviews Fetched", unit=" reviews")

# 4️⃣ 初始化翻页 token
continuation_token = None
total_fetched = 0  # 总计爬取的评论数

# 5️⃣ 循环爬取，直到没有新评论或达到 1 年前
while True:
    try:
        print(f"⏳ 正在爬取 {batch_size} 条评论...")

        # 发送爬取请求
        app_reviews, new_continuation_token = reviews(
            app_id,
            lang=target_lang,
            country=target_country,
            sort=Sort.NEWEST,  # 按最新时间排序
            count=batch_size,  # 每次最多 500 条
            continuation_token=continuation_token  # 翻页 token
        )

        # 如果没有新评论，停止爬取
        if not app_reviews:
            print("🎉 没有更多可爬取的评论，任务完成")
            break

        # 过滤掉超过 1 年前的数据
        filtered_reviews = [r for r in app_reviews if r["at"] >= one_year_ago]
        if not filtered_reviews:
            print("🎉 已经爬取到一年前的评论，任务完成")
            break

        # 更新翻页 token
        continuation_token = new_continuation_token

        # 6️⃣ **实时存储到 CSV**
        df = pd.DataFrame(filtered_reviews)

        # **每次爬取都追加存储，不检查是否重复**
        df.to_csv(output_file, mode='a', header=not pd.io.common.file_exists(output_file), index=False)

        # 更新计数
        total_fetched += len(filtered_reviews)

        print(f"💾 数据已存储到 {output_file}，当前数据量: {total_fetched}")

        # 更新 tqdm 进度条
        pbar.update(len(filtered_reviews))

        # 适当休眠，避免被封 IP
        time.sleep(5)

    except Exception as e:
        print(f"⚠️ 爬取出错：{e}")
        break

# 关闭 tqdm 进度条
pbar.close()

print(f"🎯 爬取完成，共 {total_fetched} 条评论，数据已存储至 {output_file}")


🎯 该 APP 在 Google Play 上的总评论数: 126440
⏳ 预计需要爬取 252 次，每次 500 条



Total Reviews Fetched:   3%|█▍                                             | 4000/126440 [01:08<34:45, 58.70 reviews/s]

⏳ 正在爬取 500 条评论...




Total Reviews Fetched:   0%|▏                                             | 500/126440 [00:00<01:02, 2012.00 reviews/s]

💾 数据已存储到 genshin_impact_reviews_last_year.csv，当前数据量: 500
⏳ 正在爬取 500 条评论...


...

💾 数据已存储到 genshin_impact_reviews_last_year.csv，当前数据量: 60500
⏳ 正在爬取 500 条评论...



Total Reviews Fetched:  48%|██████████████████████                        | 60671/126440 [11:37<16:41, 65.69 reviews/s]

💾 数据已存储到 genshin_impact_reviews_last_year.csv，当前数据量: 60671
⏳ 正在爬取 500 条评论...


Total Reviews Fetched:  48%|██████████████████████                        | 60671/126440 [11:43<12:42, 86.23 reviews/s]

🎉 已经爬取到一年前的评论，任务完成
🎯 爬取完成，共 60671 条评论，数据已存储至 genshin_impact_reviews_last_year.csv
